In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import io
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import glob
import random

from scipy.optimize import curve_fit

from astropy.io import fits
from astropy.time import Time
from astropy import units as u
import scipy.integrate as integrate
import lightkurve

from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

import math
import datetime as dt
import lightkurve.io as l
from lightkurve.io.generic import read_generic_lightcurve

In [2]:
def clean_text(str):
    s = result = ''.join([i for i in archivo[20:] if not i.isdigit()])
    t = s.replace('_curves.txt', '')
    u = t.replace('+', '')
    u = u.replace('-', '')
    if 'KsJs' in u:
        u = u.replace('KsJs', 'Ks Js')
    if u[0] == '_':
        u = u[1:]
    if 'AB' in u:
        u = u.replace('AB', '')
    return u

In [3]:
#repository like the one we have online
general_data = pd.read_csv("C:/Users/kmanj/FITS_BD_Files/Repository_Data/Liu_Repo.csv", encoding ='latin1')

In [4]:
dir = "C:/Users/kmanj/Manjavacas Research/Liu_2024/Liu2024/special/"
dir_save = "C:/Users/kmanj/FITS_BD_Files/Data/Liu_2024/Special/"

In [5]:
dwarfs_files = os.listdir(dir)
dwarf_dat = [archive for archive in dwarfs_files if archive.endswith(".txt")]
dwarf_dat

['2022_02_10_J0819-0335Js_Js_curves.txt',
 '2022_02_11_J0819+2103Js_Js_curves.txt',
 '2022_02_12_J0819-0335Js_Js_curves.txt',
 '2022_12_03_J0819-0335_Js_curves.txt',
 '2022_12_04_J0819-0335_Js_curves.txt']

In [6]:
new_name_file = []
new_name = []
date_obs = []
band_obs = []

for archivo in dwarf_dat:
    band_obs.append(clean_text(archivo))
    
for archivo in dwarf_dat:
    date_obs.append(archivo[0:10])

# Iterate over the elements of the first list.
for archivo in dwarf_dat:
    if 'W' in archivo:
        if 'WISE' in archivo:
            first_5_characteres = archivo[15:20]
        else:
            first_5_characteres = 'J' + archivo[12:16]
    # Extract the first 5 characters of the file name
    elif '2M' in archivo:
        if '2MASS' in archivo:
            first_5_characteres = archivo[16:21]
        else:
            first_5_characteres = 'J' + archivo[13:17]
    elif 'SDSS' in archivo:
        if 'SDSSJ' in archivo:
            first_5_characteres = archivo[15:20]
        else:
            first_5_characteres = 'J' + archivo[15:19]
    elif 'PSO' in archivo:
        first_5_characteres = 'J1112'
    else:
        first_5_characteres = archivo[11:16]
    
    # Iterate over the elements of the second list.
    for valor in general_data['Name_file']:
        # Extract the first 5 characters of the reference value
        valor = str(valor)
        first_5_values = valor[:5]
        
        # Compare the first 5 characters of the two elements
        if first_5_characteres == first_5_values:
            # Modify the file name to the desired format
            nuevo_nombre = valor + ".dat"
            # Add the new name to the new list
            new_name.append(valor)
            new_name_file.append(nuevo_nombre)
            # Break the inner loop once a match is found.
            break

            
new_name_file;
new_name
#date_obs;
#band_obs;

['J0819+21', 'J0819+21', 'J0819+21', 'J0819+21', 'J0819+21']

In [7]:
#lecture of the data
dwarfs_data = []
for i in range(len(new_name_file)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat[i]), comment='#', header=None, sep=' ')
    dwarfs_data.append(data)
dwarfs_data

[            0         1         2
 0    0.000000  1.001159  0.003659
 1    0.019634  1.005030  0.003659
 2    0.039293  0.999525  0.003659
 3    0.060332  1.013909  0.003659
 4    0.080010  1.013433  0.003659
 ..        ...       ...       ...
 222  4.563287  0.997176  0.003659
 223  4.582841  0.997620  0.003659
 224  4.604404  0.988299  0.003659
 225  4.623686  0.990807  0.003659
 226  4.643230  0.990882  0.003659
 
 [227 rows x 3 columns],
            0         1         2
 0   0.000000  1.048685  0.025658
 1   0.062253  1.017374  0.025658
 2   0.121812  1.020358  0.025658
 3   0.183903  0.963871  0.025658
 4   0.244604  0.994401  0.025658
 5   0.306078  1.016508  0.025658
 6   0.366041  1.022152  0.025658
 7   0.426846  1.019441  0.025658
 8   0.487224  0.968054  0.025658
 9   0.548701  1.001728  0.025658
 10  0.607971  0.990499  0.025658
 11  0.670363  0.974910  0.025658
 12  0.731005  0.984093  0.025658
 13  0.791248  1.001925  0.025658
 14  0.852983  1.004116  0.025658
 15  0.91

In [8]:
dwarfs_data;

In [9]:
Reference = 'Liu et. al 2024, RAS, 527, 3'
Telescope = 'New Technology Telescope'
Instrument = 'Son of Isaac (SOFI)'
TSTART = 59508.036876
TSTOP = 59512.157403

In [10]:
Object = ['ULASJ081918.58+210310.4' , '2MASS J08195820-0335266']
SpectralType = ['T6', 'T4']
Jmag = ['16.95 +/- 0.01', '14.991 +/- 0.044']
Hmag = ['17.28 +/- 0.03', '14.638 +/- 0.057']
Kmag = ['17.18 +/- 0.06', '14.586 +/- 0.105']
RA = [124.827541, 124.992527]
Dec = [21.05336564, -3.590735]
new_name = ['J0819+21', 'J0819-03']
date_obs = ['2022-02-11', '2022-02-12']
band_obs = ['Js_Js', 'Js']

In [16]:
lc = l.LightCurve(time = list(dwarfs_data[0][0]), flux = list(dwarfs_data[0][1]) * u.dimensionless_unscaled, flux_err = list(dwarfs_data[0][2]) * u.dimensionless_unscaled)
lc.to_fits(path = dir_save + Object[1] + '_Liu2024.fits')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001CB02B965C0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001CB02B96140>]

In [17]:
with fits.open(dir_save + Object[1] + '_Liu2024.fits', mode = 'update') as f:
    f[0].header['OBJECT'] = Object[1]
    f[0].header['SPECTYPE'] = SpectralType[1]
    f[0].header['TSTART'] = TSTART
    f[0].header['TSTOP'] = TSTOP
    f[0].header['DATE_OBS'] = date_obs[1]
    f[0].header['TIMESYS'] = 'mjd'
    f[0].header['TELESCOP'] = Telescope
    f[0].header['INSTRUME'] = Instrument
    f[0].header['FILTER'] = band_obs[0]
    f[0].header['VOREF'] = Reference
    f[0].header['RA_OBJ'] = RA[1]
    f[0].header['DEC_OBJ'] = Dec[1]
    f[0].header['JMAG'] = Jmag[1]
    f[0].header['HMAG'] = Hmag[1]
    f[0].header['KMAG'] = Kmag[1]
    f[1].header['TUNIT1'] = 'Hrs'

In [18]:
lc = l.LightCurve(time = list(dwarfs_data[1][0]), flux = list(dwarfs_data[1][1]) * u.dimensionless_unscaled, flux_err = list(dwarfs_data[1][2]) * u.dimensionless_unscaled)
lc.to_fits(path = dir_save + Object[0] + '_Liu2024.fits')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001CB02B95270>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001CB02B96C80>]

In [19]:
with fits.open(dir_save + Object[0] + '_Liu2024.fits', mode = 'update') as f:
    f[0].header['OBJECT'] = Object[0]
    f[0].header['SPECTYPE'] = SpectralType[0]
    f[0].header['TSTART'] = TSTART
    f[0].header['TSTOP'] = TSTOP
    f[0].header['DATE_OBS'] = date_obs[0]
    f[0].header['TIMESYS'] = 'mjd'
    f[0].header['TELESCOP'] = Telescope
    f[0].header['INSTRUME'] = Instrument
    f[0].header['FILTER'] = band_obs[0]
    f[0].header['VOREF'] = Reference
    f[0].header['RA_OBJ'] = RA[0]
    f[0].header['DEC_OBJ'] = Dec[0]
    f[0].header['JMAG'] = Jmag[0]
    f[0].header['HMAG'] = Hmag[0]
    f[0].header['KMAG'] = Kmag[0]
    f[1].header['TUNIT1'] = 'Hrs'

In [20]:
lc = l.LightCurve(time = list(dwarfs_data[2][0]), flux = list(dwarfs_data[2][1]) * u.dimensionless_unscaled, flux_err = list(dwarfs_data[2][2]) * u.dimensionless_unscaled)
lc.to_fits(path = dir_save + Object[1] + '_1_Liu2024.fits')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001CB02B2FF10>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001CB02B2FC10>]

In [21]:
with fits.open(dir_save + Object[1] + '_1_Liu2024.fits', mode = 'update') as f:
    f[0].header['OBJECT'] = Object[1]
    f[0].header['SPECTYPE'] = SpectralType[1]
    f[0].header['TSTART'] = TSTART
    f[0].header['TSTOP'] = TSTOP
    f[0].header['DATE_OBS'] = date_obs[1]
    f[0].header['TIMESYS'] = 'mjd'
    f[0].header['TELESCOP'] = Telescope
    f[0].header['INSTRUME'] = Instrument
    f[0].header['FILTER'] = band_obs[0]
    f[0].header['VOREF'] = Reference
    f[0].header['RA_OBJ'] = RA[1]
    f[0].header['DEC_OBJ'] = Dec[1]
    f[0].header['JMAG'] = Jmag[1]
    f[0].header['HMAG'] = Hmag[1]
    f[0].header['KMAG'] = Kmag[1]
    f[1].header['TUNIT1'] = 'Hrs'

In [22]:
lc = l.LightCurve(time = list(dwarfs_data[3][0]), flux = list(dwarfs_data[3][1]) * u.dimensionless_unscaled, flux_err = list(dwarfs_data[3][2]) * u.dimensionless_unscaled)
lc.to_fits(path = dir_save + Object[1] + '_2_Liu2024.fits')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001CB02B97730>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001CB02B2DB70>]

In [23]:
with fits.open(dir_save + Object[1] + '_2_Liu2024.fits', mode = 'update') as f:
    f[0].header['OBJECT'] = Object[1]
    f[0].header['SPECTYPE'] = SpectralType[1]
    f[0].header['TSTART'] = TSTART
    f[0].header['TSTOP'] = TSTOP
    f[0].header['DATE_OBS'] = date_obs[1]
    f[0].header['TIMESYS'] = 'mjd'
    f[0].header['TELESCOP'] = Telescope
    f[0].header['INSTRUME'] = Instrument
    f[0].header['FILTER'] = band_obs[1]
    f[0].header['VOREF'] = Reference
    f[0].header['RA_OBJ'] = RA[1]
    f[0].header['DEC_OBJ'] = Dec[1]
    f[0].header['JMAG'] = Jmag[1]
    f[0].header['HMAG'] = Hmag[1]
    f[0].header['KMAG'] = Kmag[1]
    f[1].header['TUNIT1'] = 'Hrs'

In [24]:
lc = l.LightCurve(time = list(dwarfs_data[4][0]), flux = list(dwarfs_data[4][1]) * u.dimensionless_unscaled, flux_err = list(dwarfs_data[4][2]) * u.dimensionless_unscaled)
lc.to_fits(path = dir_save + Object[1] + '_3_Liu2024.fits')

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001CB02B96D40>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001CB02BB9DB0>]

In [25]:
with fits.open(dir_save + Object[1] + '_3_Liu2024.fits', mode = 'update') as f:
    f[0].header['OBJECT'] = Object[1]
    f[0].header['SPECTYPE'] = SpectralType[1]
    f[0].header['TSTART'] = TSTART
    f[0].header['TSTOP'] = TSTOP
    f[0].header['DATE_OBS'] = date_obs[1]
    f[0].header['TIMESYS'] = 'mjd'
    f[0].header['TELESCOP'] = Telescope
    f[0].header['INSTRUME'] = Instrument
    f[0].header['FILTER'] = band_obs[1]
    f[0].header['VOREF'] = Reference
    f[0].header['RA_OBJ'] = RA[1]
    f[0].header['DEC_OBJ'] = Dec[1]
    f[0].header['JMAG'] = Jmag[1]
    f[0].header['HMAG'] = Hmag[1]
    f[0].header['KMAG'] = Kmag[1]
    f[1].header['TUNIT1'] = 'Hrs'